* Check if the implicit velocity operator $A$ is symmetric positive-definite.
* Check if there is any advantage in formulating the operators such that $D = -G^T$ and $H f = -E^T \tilde{f}$.

In [ ]:
import numpy
from scipy.sparse import csc_matrix, csr_matrix, hstack, identity
from scipy.sparse.linalg import inv

import pyibm

from helper import *

In [ ]:
%matplotlib qt

In [ ]:
pyibm.__version__

In [ ]:
config = dict(x=dict(start=-2.0, end=2.0, num_cells=20),
              y=dict(start=-2.0, end=2.0, num_cells=20))
grid = pyibm.GridBase(config=config)
gridc = pyibm.GridCellCentered(grid=grid)
gridx = pyibm.GridFaceX(grid=grid)
gridy = pyibm.GridFaceY(grid=grid)

In [ ]:
delta_kernel = pyibm.delta_roma_et_al_1999
delta_kernel_size = 2

In [ ]:
GHat = pyibm.assemble_GHat(gridc, gridx, gridy)
DHat = pyibm.assemble_DHat(gridc, gridx, gridy)

In [ ]:
MHat = pyibm.assemble_MHat(gridx, gridy)
R = pyibm.assemble_R(gridx, gridy)
RInv = inv(csc_matrix(R))
M = MHat @ RInv

In [ ]:
G = MHat @ GHat
D = DHat @ RInv

print('G:')
print_matrix_info(G)
print('D:')
print_matrix_info(D)

# Check if divergence is the opposite
# of the transpose of the gradient.
K = D + G.T
K = K.multiply(abs(K) > 1e-12) # remove extremely small values
assert K.nnz == 0

Indeed, we have $D = -G^T$ with their elements equal to $+1$ or $-1$.

In [ ]:
dx = (grid.x.end - grid.x.start) / (grid.M - 1)
dy = (grid.y.end - grid.y.start) / (grid.N - 1)

In [ ]:
radius = 0.5
xc, yc = 0.0, 0.0
ds = dx
N = int(round(2 * numpy.pi * radius / ds))
epsilon = 0.0
theta = numpy.linspace(0.0, 2 * numpy.pi, num=N + 1)[:-1] + epsilon
x, y = xc + radius * numpy.cos(theta), yc + radius * numpy.sin(theta)
body = pyibm.Body(x, y, grid=gridc)
body

In [ ]:
Op = pyibm.assemble_delta(body, gridc, gridx, gridy,
                          kernel=delta_kernel,
                          kernel_size=delta_kernel_size)

alpha = dx * dy
EHat = alpha * Op

# Another way to compute EHat.
EHat2 = Op @ R @ MHat
assert numpy.all((EHat - EHat2).data < 1e-12)

E = EHat @ RInv

beta = ds
HHat = beta / alpha * csr_matrix(EHat.T)
H = MHat @ HHat

# Spreding operator implemented in the decoupled IBPM of PetIBM.
HHat2 = csr_matrix(Op.T)

In [ ]:
plot_matrix(EHat - EHat2, axis_scaled=False, cmap='viridis');

In [ ]:
ux = 2 * numpy.ones(gridx.size)
uy = numpy.ones(gridy.size)
u = numpy.concatenate((ux, uy))

U = EHat @ u
Ux, Uy = U[::body.ndim], U[1::body.ndim]

assert numpy.all(Ux - 2 < 1e-12)
assert numpy.all(Uy - 1 < 1e-12)

In [ ]:
Fx = 2 * numpy.ones(body.size)
Fy = numpy.ones(body.size)
F = numpy.empty(body.ndim * body.size)
F[::body.ndim], F[1::body.ndim] = Fx, Fy

f = HHat @ F
fx, fy = f[:gridx.size], f[gridx.size:]

F2 = EHat @ f
Fx2, Fy2 = F2[::body.ndim], F2[1::body.ndim]

print(Fx2)
print(Fy2)

In [ ]:
Re = 100.0
LHat = 1 / Re * pyibm.assemble_LHat(gridx, gridy)
L = MHat @ LHat @ RInv

dt = 0.01
I = identity(gridx.size + gridy.size)

alpha_implicit = 0.5
AHat = 1 / dt * I - alpha_implicit * LHat

A = MHat @ AHat @ RInv
assert is_symmetric(A)

# Check if A is SPD.
assert numpy.all(numpy.linalg.eigvals(A.todense()) > 0)

In [ ]:
BN = pyibm.assemble_BN(gridx, gridy, dt=dt, N=1, L=L, M=M)
print_matrix_info(BN)
print('Condition number: ', condition_number(BN))
plot_matrix(BN, cmap='viridis');

In [ ]:
Q = hstack([G, E.T])
print_matrix_info(Q)

In [ ]:
QTBNQ = Q.T @ BN @ Q
print_matrix_info(QTBNQ)
print('Condition number: ', condition_number(QTBNQ))
plot_matrix(QTBNQ, cmap='viridis');
assert is_symmetric(QTBNQ)

In [ ]:
GTBNG = G.T @ BN @ G
print_matrix_info(GTBNG)
print('Condition number: ', condition_number(GTBNG))
plot_matrix(GTBNG, cmap='viridis')
assert is_symmetric(GTBNG)

In [ ]:
DBNG = DHat @ BN @ GHat
print_matrix_info(DBNG)
print('Condition number: ', condition_number(DBNG))
plot_matrix(abs(GTBNG + DBNG), cmap='viridis_r')
assert is_symmetric(DBNG)

In [ ]:
# Condition number of EBNH implemented in PetIBM.
for i in range(1, 3 + 1):
    print('Expansion order: ', i)
    BN_t = pyibm.assemble_BN(gridx, gridy, dt=dt, N=i, L=LHat, M=MHat)
    EBNH_t = EHat2 @ BN_t @ HHat2
    print_matrix_info(EBNH_t)
    print(condition_number(EBNH_t))
    plot_matrix(EBNH_t, cmap='viridis');

In [ ]:
# Refining the Lagrnagian mesh for a fix Eulerian grid.
# How does it affect the condition number of EBNH.
for i in range(1, 4 + 1):
    print('Resolution ratio: ', i)
    ds = dx / i
    N = int(round(2 * numpy.pi * radius / ds))
    theta = numpy.linspace(0.0, 2 * numpy.pi, num=N + 1)[:-1]
    x, y = xc + radius * numpy.cos(theta), yc + radius * numpy.sin(theta)
    body = pyibm.Body(x, y, grid=gridc)
    Op = pyibm.assemble_delta(body, gridc, gridx, gridy,
                              kernel=delta_kernel,
                              kernel_size=delta_kernel_size)
    EHat = Op @ R @ MHat
    HHat = csr_matrix(Op.T)
    BN = pyibm.assemble_BN(gridx, gridy, dt=dt, N=3, L=LHat, M=MHat)
    EBNH = EHat @ BN @ HHat
    print_matrix_info(EBNH)
    print(condition_number(EBNH))
    plot_matrix(EBNH, cmap='viridis');

In [ ]:
# Rotating the Lagrangian markers.
# How does it affect the condition number of EBNH.
num = 4
for i in range(num):
    print('Rotating case ', i)
    ds = dx
    N = int(round(2 * numpy.pi * radius / ds))
    epsilon = i * numpy.radians(45.0) / num
    theta = numpy.linspace(0.0, 2 * numpy.pi, num=N + 1)[:-1] + epsilon
    x, y = xc + radius * numpy.cos(theta), yc + radius * numpy.sin(theta)
    body = pyibm.Body(x, y, grid=gridc)
    Op = pyibm.assemble_delta(body, gridc, gridx, gridy,
                              kernel=delta_kernel,
                              kernel_size=delta_kernel_size)
    EHat = Op @ R @ MHat
    HHat = csr_matrix(Op.T)
    BN = pyibm.assemble_BN(gridx, gridy, dt=dt, N=3, L=LHat, M=MHat)
    EBNH = EHat @ BN @ HHat
    print_matrix_info(EBNH)
    print(condition_number(EBNH))
    plot_matrix(EBNH, cmap='viridis');